In [54]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from src.features import build_features

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

In [56]:
poblacion_teorica = pd.read_excel("../data/processed/proyeccion_problemas_de_salud.xlsx")

poblacion_teorica["Diagnostico"] = poblacion_teorica.Diagnostico.str.split().str[0]
poblacion_teorica = poblacion_teorica.query("Estrato == 'Pais'")

DIAGS_RELEVANTES = poblacion_teorica.Diagnostico.unique()
poblacion_teorica = poblacion_teorica.set_index("Diagnostico").sort_index()
# Selecciona solamente la poblacion prevista
poblacion_teorica = poblacion_teorica[[i for i in range(2017, 2036)]]

In [57]:
egresos_int = pd.read_csv("../data/processed/egresos_torax_mes_y_dia.csv")
egresos_int = egresos_int[egresos_int["DIAG1"].isin(DIAGS_RELEVANTES)]
egresos_int = egresos_int.query("ANO_EGRESO >= 2017")

In [58]:
egresos_pais = pd.read_csv(
    "../data/processed/ranking_nacional_egresos.csv",
    sep=";",
    encoding="latin-1",
    usecols=[
        "ANO_EGRESO",
        "ESTABLECIMIENTO_SALUD",
        "DIAG1",
        "n_egresos",
        "dias_estada_totales",
        "n_int_q",
        "n_muertos",
    ],
)

egresos_pais = egresos_pais[egresos_pais["DIAG1"].isin(DIAGS_RELEVANTES)]
egresos_pais = egresos_pais.query("ANO_EGRESO >= 2017")

## Separación de población teórica en hospitalizados y ambulatorios


In [59]:
# De los casos totales, se asume que una parte (0.8) se resuelve en hospitalizacion
RATIO_HOSPITALIZADOS = 0.8
RATIO_AMBULATORIO = 1 - RATIO_HOSPITALIZADOS

poblacion_teorica_hospitalizados = poblacion_teorica * RATIO_HOSPITALIZADOS
poblacion_teorica_ambulatorio = poblacion_teorica * RATIO_AMBULATORIO

In [60]:
tabla_dinamica_egresos_pais = pd.pivot_table(
    egresos_pais,
    index="DIAG1",
    columns="ANO_EGRESO",
    values=["n_egresos"],
    aggfunc="sum",
    fill_value=0,
).sort_index()

tabla_dinamica_egresos_int = pd.pivot_table(
    egresos_int,
    index="DIAG1",
    columns="ANO_EGRESO",
    values=["n_egresos"],
    aggfunc="sum",
    fill_value=0,
).sort_index()

In [61]:
tabla_dinamica_egresos_pais.columns = tabla_dinamica_egresos_pais.columns.droplevel(0)
tabla_dinamica_egresos_int.columns = tabla_dinamica_egresos_int.columns.droplevel(0)

## Obtención de brecha de atención del país

En este apartado se quiere saber si el país ha suplido la necesidad de atención para cada problema
de salud. Para esto, se contrastará la cantidad de casos atendidos a nivel país en una hospitalización con
los casos teóricos según la incidencia. La fórmula será:

$BrechaPorCasosTeoricos = \frac{Casos Atendidos Reales - Casos Teoricos}{Casos Teoricos}$

En el caso que la brecha sea positiva, entonces $Casos Atendidos Reales > Casos Teoricos$, y el pais
ha podido suplir la demanda sanitaria del problema de salud. En caso contrario, el país ha sido
incapaz de suplir la demanada para resolver estos problemas de salud. Además, se puede saber
cuantas veces de los casos reales faltó suplir de la demanda teórica. Por ejemplo:

$BrechaPorCasosTeoricos = 1.12$ Indica que el país ha resolvido todos los casos teóricos del problema de salud.
Además, ha resuelto un 12% más que los casos teóricos.


In [62]:
print(tabla_dinamica_egresos_pais.index == poblacion_teorica_hospitalizados.index)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True]


Los indices de ambas tablas son iguales, por lo que se pueden hacer operaciones entre dfs.


In [63]:
brecha_pais = (
    tabla_dinamica_egresos_pais - poblacion_teorica_hospitalizados.round(0)
) / poblacion_teorica_hospitalizados.round(0)

In [64]:
brecha_pais.style.format(precision=2).background_gradient(
    axis=None, vmin=-1, vmax=1, cmap="coolwarm"
)

,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035
DIAG1,,,,,,,,,,,,,,,,,,,
C33X,0.67,0.20,0.13,-0.12,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
C340,-0.84,-0.86,-0.86,-0.90,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
C341,-0.63,-0.54,-0.53,-0.71,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
C342,-0.95,-0.93,-0.95,-0.96,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
C343,-0.75,-0.72,-0.74,-0.79,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
C381,-0.77,-0.77,-0.72,-0.78,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
C384,6.80,7.80,9.40,7.60,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
C450,15.20,14.20,14.00,9.60,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
C780,-0.63,-0.63,-0.59,-0.70,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


Los resultados indican que en gran parte de los diagnósticos faltó suplir la demanada del problema
de salud (ej: C340, C341, C342, C343, etc). Los resultados se pueden dividir en distintos tipos
de
problema de salud:

1. **Cáncer de Pulmón (C33X a C782)**: En todos los diagnósticos faltó suplir la necesidad del
   país (exceptuando C384, C450 y E848). EL diagnóstico con una mayor necesidad de atención fue
   el C342, con una brecha máxima de -0.96 (faltó atender el 96% de la necesidad teórica).

2. **Patologías Valvulares (I051 a I712)**: En todos los diagnósticos faltó suplir la necesidad del
   país (exceptuando I420). El diagnóstico con una mayor necesidad de atención fue I051, con una
   brecha máxima de -0.98 (faltó atender el 98% de la necesidad teórica)


In [65]:
display(tabla_dinamica_egresos_pais[tabla_dinamica_egresos_pais.index == "I051"])
display(poblacion_teorica_hospitalizados[poblacion_teorica_hospitalizados.index == "I051"])

ANO_EGRESO,2017,2018,2019,2020
DIAG1,,,,
I051,22,39,26,26


,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035
Diagnostico,,,,,,,,,,,,,,,,,,,
I051,1124.30748,1144.585761,1166.304465,1187.735242,1201.167278,1210.335486,1218.412665,1226.072452,1233.432411,1240.504139,1247.278603,1253.741213,1259.879639,1265.682041,1271.243273,1276.485449,1281.402404,1285.990476,1290.24942


Al observar el diagnóstico I051, se puede apreciar que en el año 2020 se tenía una población teórica
de 1188. Sin embargo, solamente se atendieron 26 pacientes en ese año.

Otra métrica altamente relevantes es la Brecha con respecto a los casos atendidos realmente. Esta métrica permitira determinar cuántas veces más debe crecer el país para suplir la demanda teórica. El cálculo de esta métrica es:

$BrechaPorCasosReales = \frac{Casos Atendidos Reales - Casos Teoricos}{Casos Atendidos Reales}$

Si $BrechaPorCasosReales = -2.34$, entonces el país debe crecer un **234%** para suplir la necesidad teórica del problema de salud.


In [66]:
brecha_pais_por_egresos_reales = (
    tabla_dinamica_egresos_pais - poblacion_teorica_hospitalizados.round(0)
) / tabla_dinamica_egresos_pais

In [67]:
brecha_pais_por_egresos_reales.style.format(precision=2).background_gradient(
    axis=None, vmin=-1, vmax=1, cmap="coolwarm"
)

,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035
DIAG1,,,,,,,,,,,,,,,,,,,
C33X,0.40,0.17,0.12,-0.14,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
C340,-5.24,-6.24,-6.13,-9.43,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
C341,-1.73,-1.17,-1.14,-2.40,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
C342,-19.34,-14.22,-19.29,-21.38,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
C343,-2.94,-2.53,-2.78,-3.82,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
C381,-3.32,-3.39,-2.52,-3.45,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
C384,0.87,0.89,0.90,0.88,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
C450,0.94,0.93,0.93,0.91,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
C780,-1.73,-1.73,-1.45,-2.29,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


En la tabla anterior se puede apreciar los diagnosticos en donde ha habido una falta de atención
a nivel país. Con estos resultados es posible calcular la brecha que se quiere hacer cargo el
INT del Tórax para cada uno de los problemas de salud.

A modo de ejemplo, se observaran los resultados del diagnóstico **J848 - Otras enfermedades pulmonares intersticiales especificadas**


In [68]:
tabla_resumen_ppt = build_features.obtener_tabla_resumen_egresos(
    tabla_dinamica_egresos_int,
    tabla_dinamica_egresos_pais,
    poblacion_teorica_hospitalizados,
    brecha_pais_por_egresos_reales,
)

In [69]:
display(tabla_resumen_ppt[tabla_resumen_ppt.index == "J848"])

ANO_EGRESO,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035
DIAG1,,,,,,,,,,,,,,,,,,,
J848,25; 113; 4244; (-36.56),53; 172; 4320; (-24.12),52; 158; 4402; (-26.86),27; 107; 4483; (-40.9),0; 0; 4534; (-),0; 0; 4569; (-),0; 0; 4599; (-),0; 0; 4628; (-),0; 0; 4656; (-),0; 0; 4682; (-),0; 0; 4708; (-),0; 0; 4732; (-),0; 0; 4756; (-),0; 0; 4777; (-),0; 0; 4798; (-),0; 0; 4818; (-),0; 0; 4837; (-),0; 0; 4854; (-),0; 0; 4870; (-)


Los resultados indican que en 2020 el país atendió 107 casos del diagnósticos J848. Sin embargo,
teóricamente debieron haber 4483 casos, por lo que el país debe crecer un 4090% de lo que produce
actualmente para tal problema de salud. Además, se muestra la cantidad de egresos del INT,
siendo este 27 para el año 2020.

Ahora, es importante saber cuánto va a crecer el INT en este problema de salud. Esto, ya que es
imposible que el recinto quiera resolver el total de la brecha calculada.


In [70]:
tabla_resumen_ppt.iloc[:5, :4]

ANO_EGRESO,2017,2018,2019,2020
DIAG1,,,,
C33X,1; 25; 15; (0.4),1; 18; 15; (0.17),4; 17; 15; (0.12),7; 14; 16; (-0.14)
C340,21; 163; 1017; (-5.24),17; 143; 1035; (-6.24),24; 148; 1055; (-6.13),7; 103; 1074; (-9.43)
C341,180; 372; 1017; (-1.73),220; 478; 1035; (-1.17),220; 493; 1055; (-1.14),76; 316; 1074; (-2.4)
C342,19; 50; 1017; (-19.34),24; 68; 1035; (-14.22),20; 52; 1055; (-19.29),7; 48; 1074; (-21.38)
C343,118; 258; 1017; (-2.94),124; 293; 1035; (-2.53),120; 279; 1055; (-2.78),38; 223; 1074; (-3.82)


In [71]:
brecha_total = tabla_dinamica_egresos_pais - poblacion_teorica_hospitalizados.round(0)

# Solamente se dejan las brechas que sean negativas. Las positivas es porque se atendieron
# completamente
brecha_total[brecha_total > 0] = 0

In [72]:
BRECHA_QUE_TORAX_ATENDERA = 0.1
casos_nuevos_del_torax = brecha_total * BRECHA_QUE_TORAX_ATENDERA * -1

In [75]:
casos_finales_torax_con_brecha = tabla_dinamica_egresos_int + casos_nuevos_del_torax
porcentaje_crecimiento_torax = (
    (casos_finales_torax_con_brecha / tabla_dinamica_egresos_int) - 1
) * 100
porcentaje_crecimiento_torax = porcentaje_crecimiento_torax.dropna(axis=1, how="all")

In [76]:
display(
    porcentaje_crecimiento_torax.style.format(precision=2).background_gradient(
        axis=None, vmin=0, vmax=2000, cmap="Blues"
    )
)

,2017,2018,2019,2020
DIAG1,,,,
C33X,0.00,0.00,0.00,2.86
C340,406.67,524.71,377.92,1387.14
C341,35.83,25.32,25.55,99.74
C342,508.95,402.92,501.50,1465.71
C343,64.32,59.84,64.67,223.95
C381,90.67,66.19,68.95,290.00
C384,0.00,0.00,0.00,0.00
C450,0.00,0.00,0.00,0.00
C780,112.98,104.13,100.81,498.67
